In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:-2]:
  layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

model.fit(train_generator,
          epochs=10,
          validation_data=validation_generator)

model.save('final_model.h5')



Found 109 images belonging to 2 classes.
Found 109 images belonging to 2 classes.
Epoch 1/10
4/4 [==============================] - 169s 44s/step - loss: 7.2992 - accuracy: 0.4862 - val_loss: 2.6067 - val_accuracy: 0.4587
Epoch 2/10
4/4 [==============================] - 163s 43s/step - loss: 1.3249 - accuracy: 0.5321 - val_loss: 0.6882 - val_accuracy: 0.5688
Epoch 3/10
4/4 [==============================] - 158s 48s/step - loss: 0.6289 - accuracy: 0.6514 - val_loss: 0.5210 - val_accuracy: 0.7339
Epoch 4/10
4/4 [==============================] - 164s 44s/step - loss: 0.4667 - accuracy: 0.7798 - val_loss: 0.3377 - val_accuracy: 0.8807
Epoch 5/10
4/4 [==============================] - 170s 44s/step - loss: 0.4203 - accuracy: 0.7890 - val_loss: 0.2319 - val_accuracy: 0.8899
Epoch 6/10
4/4 [==============================] - 163s 43s/step - loss: 0.3065 - accuracy: 0.8716 - val_loss: 0.1779 - val_accuracy: 0.9266
Epoch 7/10
4/4 [==============================] - 161s 42s/step - loss: 0.2202

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [27]:
# from tensorflow.keras.models import load_model

# loaded_model = load_model("final_model.h5")
# def predict_cleanliness(img_path, model):
#     img = cv2.imread(img_path)
#     img = cv2.resize(img, (224, 224))  # Reshape to match model input
#     img = img.astype("float32") / 255.0  # Normalize pixel values
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     pred = model.predict(img)[0][0]
#     print(pred)
#     if pred > 0.9:
#         return "Clean"
#     else:
#         return "Messy"

# new_image = "/content/clean.jpg"
# prediction = predict_cleanliness(new_image, loaded_model)
# print("Predicted cleanliness:", prediction)


1/1 [==============================] - 1s 634ms/step
0.7983789
Predicted cleanliness: Messy


In [39]:
from tensorflow.keras.models import load_model
from PIL import Image
from pillow_heif import register_heif_opener
register_heif_opener()
import numpy as np

loaded_model = load_model("final_model.h5")

def predict_cleanliness(img_path, model):
  try:
      img = Image.open(img_path)
      img = img.convert('RGB')
      img = img.resize((224, 224))
      img_data = np.array(img)
      img_data = img_data.astype("float32") / 255.0
      img_data = np.expand_dims(img_data, axis=0)
      pred = model.predict(img_data)[0][0]
      print(pred)
      if pred > 0.9:
          return "Messy"
      else:
          return "Clean"
  except Exception as e:
      return (f"Error reading image {img_path}: {e}")

# Example usage with the loaded model
new_image = "/content/IMG_0885.HEIC"  # Or a HEIF image path
prediction = predict_cleanliness(new_image, loaded_model)
print("Predicted cleanliness:", prediction)


1/1 [==============================] - 1s 604ms/step
0.5538965
Predicted cleanliness: Clean
